In [1]:
import os

%pwd # print current directory

'/Users/wilsvenleong/Downloads/learning-materials/chicken-disease-classification/research'

In [2]:
os.chdir("../")  # Move up to root directory
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/chicken-disease-classification'

## 1. Update `params.yaml` and `config.yaml`

## 4. Update the Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class CallbacksConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoints_file_path: Path

## 5. Update the `ConfigurationManager` in `src/config/configuration.py`

In [4]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_callbacks_config(self) -> CallbacksConfig:
        callbacks_preparation = self.config.callbacks_preparation

        create_directories(
            [callbacks_preparation.root_dir, callbacks_preparation.tensorboard_log_dir]
        )

        callbacks_config = CallbacksConfig(
            root_dir=Path(callbacks_preparation.root_dir),
            tensorboard_log_dir=Path(callbacks_preparation.tensorboard_log_dir),
            checkpoints_file_path=Path(callbacks_preparation.checkpoints_file_path),
        )

        return callbacks_config

## 6. Update the Components

In [5]:
import time
import tensorflow as tf


class Callbacks:
    def __init__(self, config: CallbacksConfig):
        self.callbacks_config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.callbacks_config.tensorboard_log_dir, f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.callbacks_config.checkpoints_file_path), save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]

## 7. Update the Pipeline

In [6]:
try:
    config_manager = ConfigurationManager()
    callbacks_config = config_manager.get_callbacks_config()
    callbacks = Callbacks(config=callbacks_config)
    callbacks_list = callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[ 2023-09-22 21:48:07,681 ] 33 common chicken-disease-logger -  INFO - Loaded YAML file successfully from: config/config.yaml
[ 2023-09-22 21:48:07,687 ] 33 common chicken-disease-logger -  INFO - Loaded YAML file successfully from: params.yaml
[ 2023-09-22 21:48:07,689 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts
[ 2023-09-22 21:48:07,692 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts/callbacks
[ 2023-09-22 21:48:07,693 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts/callbacks/tensorboard_logs


## Example

In [7]:
import time


timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-09-22-21-56-03'